## Import packages

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

## Load joint location

In [2]:
def load_joint_location(file_path):
    df = pd.read_csv(file_path)
    columns = []
    columns.append('frame_id')
    for e in list(df.columns):
        if 'p0' in e:
            columns.append(e)
    f_df = df.filter(items = columns)
    return f_df

In [3]:
vertex_to_joints = {
    'left_shoulder': ['left_elbow', 'left_shoulder', 'left_hip'],
    'right_shoulder': ['right_elbow', 'right_shoulder', 'right_hip'],
    'left_elbow': ['left_wrist', 'left_elbow', 'left_shoulder'],
    'right_elbow': ['right_wrist', 'right_elbow', 'right_shoulder'],
    'left_hip': ['left_knee', 'left_hip', 'right_hip'],
    'right_hip': ['right_knee', 'right_hip', 'left_hip'],
    'left_knee': ['left_ankle', 'left_knee', 'left_hip'],
    'right_knee': ['right_ankle', 'right_knee', 'right_hip'],
}

In [4]:
def get_vertex_joint_location(vertex, vertex_to_joints, joint_location):
    joints = vertex_to_joints[vertex]
    columns = []
    for joint in joints:
        columns.append('p0_{}_x'.format(joint))
        columns.append('p0_{}_y'.format(joint))
    f_df = joint_location.filter(items=columns)
    return f_df

In [5]:
def measure_angle(XY):
    
    x1, y1, x2, y2, x3, y3 = XY
    
    v1 = np.array([x1 - x2, y1 - y2])
    v2 = np.array([x3 - x2, y3 - y2])
    cos = np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))
    theta = np.arccos(cos)
    theta = theta * 180 / np.pi
    
    return theta

In [6]:
def drop_prefix(self, prefix):
    self.columns = self.columns.str.lstrip(prefix)
    return self

pd.core.frame.DataFrame.drop_prefix = drop_prefix

In [7]:
columns = ['left_shoulder',
    'right_shoulder',
    'left_elbow',
    'right_elbow',
    'left_hip',
    'right_hip',
    'left_knee',
    'right_knee']


In [8]:
def create_distribution(benign_joint_location):
    hist_array = []
    columninfo = []
    for col in benign_joint_location.columns:
        if col != 'frame_id':
            hist, bin_edges = np.histogram(benign_joint_location[col], bins = 10)
            min = np.min(benign_joint_location[col])
            max = np.max(benign_joint_location[col])
            step = bin_edges[1] - bin_edges[0]
            columninfo.append((min, max, step))
            hist_array.append(hist)

    output_df = pd.DataFrame(columns, columns = ['Joints'])
    hist_array = np.array(hist_array).T
    columninfo = np.array(columninfo).T
    
    for i in range(10):
        output_df[i] = hist_array[i]

    output_df['Min'] = columninfo[0]
    output_df['Max'] = columninfo[1]
    output_df['Step'] = columninfo[2]

    return output_df    

In [10]:
folder = '/nethome/abhagavatula8/skeleton-vis/public/pgd_interpolated_data'
for dirname, dirs, files in os.walk(folder):
    for file in files:
        if 'keypoints-' in file:
            input_file_path = os.path.join(dirname, file)
            benign_joint_location = load_joint_location(input_file_path)
            output_df = benign_joint_location["frame_id"]
            output_df = output_df.to_frame()            

            #format 1 
            for column in columns:
                output_df[column] = get_vertex_joint_location(column, vertex_to_joints, benign_joint_location).apply(measure_angle, axis= 1)
            output_filename = file.replace("keypoints", "angles")
            output_file_path = os.path.join(dirname, output_filename)
            output_df.to_csv(output_file_path, index= False)

            #format 2 
            for column in columns:
                output_df[column] = output_df[column] - output_df[column].shift(-1)
            output_filename = file.replace("keypoints", "angle-changes")
            output_file_path = os.path.join(dirname, output_filename)
            output_df.to_csv(output_file_path, index= False)

        

        #format 3 
        if 'displacements-' in file:
            #displacement distribution
            input_file_path = os.path.join(dirname, file)
            benign_joint_location = load_joint_location(input_file_path)

            #remove extra columns
            benign_joint_location.drop_prefix('p0_')
            delete = []
            removable = ['eye', 'nose', 'ear', 'ankle', 'wrist', 'average']
            for col in benign_joint_location.columns:
                if any(s in col for s in removable):
                    delete.append(col)
            benign_joint_location = benign_joint_location.drop(delete, axis= 1)
            output_df = create_distribution(benign_joint_location)

            output_filename = file.replace("displacements", "distribution")
            output_file_path = os.path.join(dirname, output_filename)
            output_df.to_csv(output_file_path, index= False)

            #average of displacements
            average_df = benign_joint_location
            average_df['average'] = average_df[average_df.columns[1:]].mean(axis = 1)
            output_filename = file.replace("displacements", "average")
            output_file_path = os.path.join(dirname, output_filename)
            average_df.to_csv(output_file_path, index = False)
        
        
        #format 4
        if 'angles' in file:
            input_file_path = os.path.join(dirname, file)
            benign_joint_location = pd.read_csv(input_file_path)
            benign_joint_location = benign_joint_location.fillna(0)
            output_df = create_distribution(benign_joint_location)

            output_filename = file.replace("angles", "a-distribution")
            output_file_path = os.path.join(dirname, output_filename)
            output_df.to_csv(output_file_path, index= False)


        #format 5
        if 'angle-changes' in file:
            input_file_path = os.path.join(dirname, file)
            benign_joint_location = pd.read_csv(input_file_path)
            benign_joint_location = benign_joint_location.fillna(0)
            output_df = create_distribution(benign_joint_location)

            output_filename = file.replace("angle", "a-c-distribution")
            output_file_path = os.path.join(dirname, output_filename)
            output_df.to_csv(output_file_path, index= False)


#benign_joint_location = load_joint_location('/home/aiswarya/skeleton-vis/public/data/item-00051/keypoints-benign.csv')
#adv_joint_location = load_joint_location('/home/aiswarya/skeleton-vis/public/data/item-00051/keypoints-attacked-eps_001.csv')

## Get joint locations for a vertex

In [10]:
#benign_vertex_joint_location = get_vertex_displacement('left_knee', vertex_to_joints, benign_joint_location)

In [11]:
#benign_vertex_joint_location.head()

## Measure angle of three points

In [12]:
measure_angle([6, 0, 0, 0, 0, 6])

90.0

## Measure angle of all frames

In [13]:
#left_knee_angles = benign_vertex_joint_location.apply(measure_angle, axis=1)
#left_knee_angles

In [14]:

#X = np.arange(0, len(left_knee_angles))
#plt.plot(X, left_knee_angles)
#plt.title('left_knee')
#plt.xlabel('frame')
#plt.ylabel('degree')
#plt.show()

## Show all results

In [15]:

#for vertex in vertex_to_joints:
#    
#    benign_vertex_joint_location = get_vertex_joint_location(
#        vertex, 
#        vertex_to_joints, 
#        benign_joint_location
#    )
#    
#    adv_vertex_joint_location = get_vertex_joint_location(
#        vertex, 
#        vertex_to_joints, 
#        adv_joint_location
#    )
#
#    benign_angles = benign_vertex_joint_location.apply(measure_angle, axis=1)
#    adv_angles = adv_vertex_joint_location.apply(measure_angle, axis=1)
#    
#    X = np.arange(0, len(benign_angles))
#    
#    plt.plot(X, benign_angles, label='benign')
#    plt.plot(X, adv_angles, color='orange', label='attacked')
#    plt.title(vertex)
#    plt.xlabel('frame')
#    plt.ylabel('degree')
#    plt.legend()
#    plt.show()